In [21]:
import os
import pandas as pd
import numpy as np

In [22]:
folder = "./resources/"
review_df = pd.read_csv(f"{folder}media_customer_reviews.csv")
gold_review_df = pd.read_csv(f"{folder}media_gold_reviews_chunked.csv")
customer_df = pd.read_csv(f"{folder}sales_customers.csv")
franchises_df = pd.read_csv(f"{folder}sales_franchises.csv")
suppliers_df = pd.read_csv(f"{folder}sales_suppliers.csv")
transactions_df = pd.read_csv(f"{folder}sales_transactions.csv")
suppliers_df_add = pd.read_csv(f"{folder}sales_suppliers_add.csv")


## extra_suppliers

In [23]:
suppliers_df_add.head()

,supplier_id,name,ingredient,continent,city,district,size,longitude,latitude,approved
0,4000027,Amazonia Cocoa,cacao,South America,Manaus,Centro,L,-60.0217,-3.1190,True
1,4000028,Island Palms,coconut,Asia,Phuket,Old Town,M,98.3923,7.8804,True
2,4000029,Iberian Nuts,almonds,Europe,Seville,Triana,S,-5.9845,37.3891,True
3,4000030,Cane Fields,cane sugar,South America,Recife,Boa Viagem,XL,-34.8811,-8.0578,True
4,4000031,Veracruz Orchid,vanilla,North America,Veracruz,Reforma,M,-96.1342,19.1738,True


In [24]:
suppliers_df_add.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   supplier_id  21 non-null     int64  
 1   name         21 non-null     object 
 2   ingredient   21 non-null     object 
 3   continent    21 non-null     object 
 4   city         21 non-null     object 
 5   district     21 non-null     object 
 6   size         21 non-null     object 
 7   longitude    21 non-null     float64
 8   latitude     21 non-null     float64
 9   approved     21 non-null     bool   
dtypes: bool(1), float64(2), int64(1), object(6)
memory usage: 1.6+ KB


In [25]:
suppliers_df_add = suppliers_df_add.astype({
    col: "string" for col in suppliers_df_add.columns
    if col != "supplier_id" and col != "latitude" and col != "longitude" and col != "approved"
})
# suppliers_df_add = suppliers_df_add.astype({"latitude": "float64", "longitude": "float64", "approved": "boolean"})

suppliers_df_add.info()
suppliers_df_add.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   supplier_id  21 non-null     int64  
 1   name         21 non-null     string 
 2   ingredient   21 non-null     string 
 3   continent    21 non-null     string 
 4   city         21 non-null     string 
 5   district     21 non-null     string 
 6   size         21 non-null     string 
 7   longitude    21 non-null     float64
 8   latitude     21 non-null     float64
 9   approved     21 non-null     bool   
dtypes: bool(1), float64(2), int64(1), string(6)
memory usage: 1.6 KB


supplier_id    0
name           0
ingredient     0
continent      0
city           0
district       0
size           0
longitude      0
latitude       0
approved       0
dtype: int64

## transactions dataframe

In [4]:
transactions_df.head()

,transactionID,customerID,franchiseID,dateTime,product,quantity,unitPrice,totalPrice,paymentMethod,cardNumber
0,2002961,1000253,3000047,2024-05-14T12:17:01.495Z,Golden Gate Ginger,8,3,24,amex,378154478982993
1,2003007,1000226,3000047,2024-05-10T23:10:10.239Z,Austin Almond Biscotti,36,3,108,mastercard,2244626981238094
2,2003017,1000108,3000047,2024-05-16T16:34:10.613Z,Austin Almond Biscotti,40,3,120,mastercard,2490570234487424
3,2003068,1000173,3000047,2024-05-02T04:31:51.612Z,Pearly Pies,28,3,84,amex,343808569426192
4,2003103,1000075,3000047,2024-05-04T23:44:26.902Z,Pearly Pies,28,3,84,visa,4377080942201798


In [5]:
transactions_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   transactionID  3333 non-null   int64 
 1   customerID     3333 non-null   int64 
 2   franchiseID    3333 non-null   int64 
 3   dateTime       3333 non-null   object
 4   product        3333 non-null   object
 5   quantity       3333 non-null   int64 
 6   unitPrice      3333 non-null   int64 
 7   totalPrice     3333 non-null   int64 
 8   paymentMethod  3333 non-null   object
 9   cardNumber     3333 non-null   int64 
dtypes: int64(7), object(3)
memory usage: 260.5+ KB


In [6]:
transactions_df = transactions_df.rename(columns={
    "transactionID": "transaction_id",
    "customerID": "customer_id",
    "franchiseID": "franchise_id",
    "dateTime": "date_time",
    "unitPrice": "unit_price",
    "totalPrice": "total_price",
    "paymentMethod": "payment_method",
    "cardNumber": "card_number",
})
transactions_df.columns = transactions_df.columns.str.lower()

transactions_df['date_time'] = pd.to_datetime(transactions_df['date_time'])

transactions_df = transactions_df.astype({
    "transaction_id": "int64",
    "customer_id": "int64",
    "franchise_id": "int64",
    "product": "string",
    "quantity": "int64",
    "unit_price": "float64",
    "total_price": "float64",
    "payment_method": "string",
    "card_number": "string",
})

transactions_df.info()
transactions_df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   transaction_id  3333 non-null   int64              
 1   customer_id     3333 non-null   int64              
 2   franchise_id    3333 non-null   int64              
 3   date_time       3333 non-null   datetime64[ns, UTC]
 4   product         3333 non-null   string             
 5   quantity        3333 non-null   int64              
 6   unit_price      3333 non-null   float64            
 7   total_price     3333 non-null   float64            
 8   payment_method  3333 non-null   string             
 9   card_number     3333 non-null   string             
dtypes: datetime64[ns, UTC](1), float64(2), int64(4), string(3)
memory usage: 260.5 KB


transaction_id    0
customer_id       0
franchise_id      0
date_time         0
product           0
quantity          0
unit_price        0
total_price       0
payment_method    0
card_number       0
dtype: int64

## suppliers dataframe

In [27]:
suppliers_df.sample(10)

,supplierID,name,ingredient,continent,city,district,size,longitude,latitude,approved
12,4000012,Coffee Collective,coffee,South America,Medellin,El Poblado,XXL,-75.5638,6.2518,Y
1,4000001,Coconut Grove,coconut,Asia,Manila,Intramuros,S,121.0221,14.6042,Y
15,4000015,Honey Hives,honey,Africa,Addis Ababa,Piazza,S,38.7575,9.0084,Y
8,4000008,Cashew Corner,cashews,Asia,Goa,Anjuna Beach,XL,73.8067,15.3173,Y
19,4000019,Cardamom Cove,cardamom,Asia,Kozhikode,Mappila Bay,XL,75.7804,11.2588,Y
16,4000016,Sesame Seeds,sesame seeds,Asia,Yangon,Chinatown,L,96.1611,16.8409,Y
25,4000025,Nutmeg Nirvana,nutmeg,Asia,Banda Aceh,Peunayong,M,95.3198,5.5577,Y
13,4000013,Ginger Gems,ginger,Asia,Chennai,Mylapore,M,80.2707,13.0827,Y
5,4000005,Pecan Pleasures,pecans,North America,Atlanta,Virginia-Highland,S,-84.3888,33.7490,Y
9,4000009,Maple Monarch,maple syrup,North America,Montreal,Plateau Mont-Royal,M,-73.5673,45.5017,Y


In [28]:
suppliers_df = suppliers_df.rename(columns={'supplierID': 'supplier_id'})
suppliers_df.columns = suppliers_df.columns.str.lower()

suppliers_df["approved"] = suppliers_df["approved"].map({"Y": True, "N": False})
suppliers_df = suppliers_df.astype({
    col: "string" for col in suppliers_df.columns
    if col != "supplier_id" and col != "latitude" and col != "longitude" and col != "approved"
})
suppliers_df = suppliers_df.astype({"latitude": "float64", "longitude": "float64", "approved": "boolean"})

suppliers_df.info()
suppliers_df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   supplier_id  27 non-null     int64  
 1   name         27 non-null     string 
 2   ingredient   27 non-null     string 
 3   continent    27 non-null     string 
 4   city         27 non-null     string 
 5   district     27 non-null     string 
 6   size         27 non-null     string 
 7   longitude    27 non-null     float64
 8   latitude     27 non-null     float64
 9   approved     27 non-null     boolean
dtypes: boolean(1), float64(2), int64(1), string(6)
memory usage: 2.1 KB


supplier_id    0
name           0
ingredient     0
continent      0
city           0
district       0
size           0
longitude      0
latitude       0
approved       0
dtype: int64

## franchises dataframe

In [9]:
franchises_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   franchiseID  48 non-null     int64  
 1   name         48 non-null     object 
 2   city         48 non-null     object 
 3   district     48 non-null     object 
 4   zipcode      48 non-null     object 
 5   country      48 non-null     object 
 6   size         48 non-null     object 
 7   longitude    48 non-null     float64
 8   latitude     48 non-null     float64
 9   supplierID   48 non-null     int64  
dtypes: float64(2), int64(2), object(6)
memory usage: 3.9+ KB


In [10]:
franchises_df = franchises_df.rename(columns={'franchiseID': 'franchise_id', "supplierID": "supplier_id"})
franchises_df.columns = franchises_df.columns.str.lower()

franchises_df = franchises_df.astype({
    col: "string" for col in franchises_df.columns
    if col != "franchise_id" and col != "supplier_id" and col != "longitude" and col != "latitude"
})


franchises_df.info()

franchises_df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   franchise_id  48 non-null     int64  
 1   name          48 non-null     string 
 2   city          48 non-null     string 
 3   district      48 non-null     string 
 4   zipcode       48 non-null     string 
 5   country       48 non-null     string 
 6   size          48 non-null     string 
 7   longitude     48 non-null     float64
 8   latitude      48 non-null     float64
 9   supplier_id   48 non-null     int64  
dtypes: float64(2), int64(2), string(6)
memory usage: 3.9 KB


franchise_id    0
name            0
city            0
district        0
zipcode         0
country         0
size            0
longitude       0
latitude        0
supplier_id     0
dtype: int64

## customer dataframe

In [11]:
customer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   customerID       300 non-null    int64 
 1   first_name       300 non-null    object
 2   last_name        300 non-null    object
 3   email_address    300 non-null    object
 4   phone_number     300 non-null    object
 5   address          300 non-null    object
 6   city             300 non-null    object
 7   state            300 non-null    object
 8   country          300 non-null    object
 9   continent        300 non-null    object
 10  postal_zip_code  300 non-null    int64 
 11  gender           300 non-null    object
dtypes: int64(2), object(10)
memory usage: 28.3+ KB


In [12]:
customer_df = customer_df.rename(columns={'customerID': 'customer_id'})
customer_df.columns = customer_df.columns.str.lower()
customer_df = customer_df.astype({
    col: "string" for col in customer_df.columns
    if col != "customer_id"
})

customer_df.info()
# check null
customer_df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   customer_id      300 non-null    int64 
 1   first_name       300 non-null    string
 2   last_name        300 non-null    string
 3   email_address    300 non-null    string
 4   phone_number     300 non-null    string
 5   address          300 non-null    string
 6   city             300 non-null    string
 7   state            300 non-null    string
 8   country          300 non-null    string
 9   continent        300 non-null    string
 10  postal_zip_code  300 non-null    string
 11  gender           300 non-null    string
dtypes: int64(1), string(11)
memory usage: 28.3 KB


customer_id        0
first_name         0
last_name          0
email_address      0
phone_number       0
address            0
city               0
state              0
country            0
continent          0
postal_zip_code    0
gender             0
dtype: int64

## gold review dataframe

In [13]:
gold_review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   franchiseID   196 non-null    int64 
 1   review_date   196 non-null    object
 2   chunked_text  196 non-null    object
 3   chunk_id      196 non-null    object
 4   review_uri    196 non-null    object
dtypes: int64(1), object(4)
memory usage: 7.8+ KB


In [14]:
# change column name
gold_review_df = gold_review_df.rename(columns={'franchiseID': 'franchise_id'})
# change column to lower case
gold_review_df.columns = gold_review_df.columns.str.lower()
# change column type
gold_review_df = gold_review_df.astype({'chunked_text': 'string', 'chunk_id': 'string', 'review_uri': 'string'})
gold_review_df['review_date'] = pd.to_datetime(gold_review_df['review_date'])
# check info
gold_review_df.info()
# check null
gold_review_df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196 entries, 0 to 195
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   franchise_id  196 non-null    int64              
 1   review_date   196 non-null    datetime64[ns, UTC]
 2   chunked_text  196 non-null    string             
 3   chunk_id      196 non-null    string             
 4   review_uri    196 non-null    string             
dtypes: datetime64[ns, UTC](1), int64(1), string(3)
memory usage: 7.8 KB


franchise_id    0
review_date     0
chunked_text    0
chunk_id        0
review_uri      0
dtype: int64

## review dataframe

In [15]:
review_df.head()

,review,franchiseID,review_date,new_id
0,Title: A Delightful Cookie Experience at Bakeh...,3000037,2024-05-20T17:24:06.591Z,1
1,"""Sweet tooth heaven on East 6th Street! I'm ob...",3000017,2024-05-20T17:17:03.052Z,2
2,**4.5/5 stars**\n\nI stumbled upon Bakehouse i...,3000007,2024-05-20T17:17:03.052Z,3
3,"Bakehouse in Fitzroy, Melbourne, has disappoin...",3000003,2024-05-31T15:13:36.962Z,4
4,Title: A Sweet Escape in Las Vegas' Arts Distr...,3000034,2024-05-20T17:24:06.591Z,5


In [16]:
review_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   review       204 non-null    object
 1   franchiseID  204 non-null    int64 
 2   review_date  204 non-null    object
 3   new_id       204 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 6.5+ KB


In [17]:
# change column name
review_df = review_df.rename(columns={'franchiseID': 'franchise_id'})
# change column to lower case
review_df.columns = review_df.columns.str.lower()
# change column type
review_df = review_df.astype({'review': 'string'})
review_df['review_date'] = pd.to_datetime(review_df['review_date'])
# organise column order
review_df = review_df[['new_id', 'franchise_id', 'review_date', 'review']]
# check info
review_df.info()
# check null
review_df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   new_id        204 non-null    int64              
 1   franchise_id  204 non-null    int64              
 2   review_date   204 non-null    datetime64[ns, UTC]
 3   review        204 non-null    string             
dtypes: datetime64[ns, UTC](1), int64(2), string(1)
memory usage: 6.5 KB


new_id          0
franchise_id    0
review_date     0
review          0
dtype: int64

## save file

In [29]:
merge_suppliers = pd.concat([suppliers_df, suppliers_df_add])
merge_suppliers.head()
merge_suppliers.info()
merge_suppliers.isnull().sum()
merge_suppliers.to_csv(f"./cleaned_resources/sales_suppliers.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 48 entries, 0 to 20
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   supplier_id  48 non-null     int64  
 1   name         48 non-null     string 
 2   ingredient   48 non-null     string 
 3   continent    48 non-null     string 
 4   city         48 non-null     string 
 5   district     48 non-null     string 
 6   size         48 non-null     string 
 7   longitude    48 non-null     float64
 8   latitude     48 non-null     float64
 9   approved     48 non-null     boolean
dtypes: boolean(1), float64(2), int64(1), string(6)
memory usage: 3.8 KB


In [20]:
folder = "./cleaned_resources/"
review_df = review_df.to_csv(f"{folder}media_customer_reviews.csv", index=False)
gold_review_df = gold_review_df.to_csv(f"{folder}media_gold_reviews_chunked.csv", index=False)
customer_df = customer_df.to_csv(f"{folder}sales_customers.csv", index=False)
franchises_df = franchises_df.to_csv(f"{folder}sales_franchises.csv", index=False)
suppliers_df = suppliers_df.to_csv(f"{folder}sales_suppliers.csv", index=False)
transactions_df = transactions_df.to_csv(f"{folder}sales_transactions.csv", index=False)